In [1]:
import modern_robotics as mr
import numpy as np
from calibration import *
np.set_printoptions(precision=16)

In [2]:
s = 0.001 # using mm as units


w1 = np.array([1,0,0])
q1 = np.array([0,0,0])*s
v1 = mr.ScrewToAxis(q1, w1, 0)

w2 = np.array([1,0,0])
q2 = np.array([0,0,1250])*s
v2 = mr.ScrewToAxis(q2, w2, 0)

w3 = np.array([0,1,0])
q3 = np.array([0,0,1250+45])*s
v3 = mr.ScrewToAxis(q3, w3, 0)

w4 = np.array([1,0,0])
q4 = np.array([0,740+360,1250+45])*s
v4 = mr.ScrewToAxis(q4, w4, 0)

w5 = np.array([0,1,0])
q5 = np.array([0,0,1250+45])*s
v5 = mr.ScrewToAxis(q5, w5, 0)


xi0 = np.c_[v1, v2, v3, v4, v5]


In [5]:
wist0 = np.array([0,0,0])
vist0 = np.array([0,1310,1295])*s

xist0 = np.r_[wist0, vist0]


# introducing some random error

w01=np.array([0.9989, -0.0199, 0.0209])
q01=np.array([0.0020, 0.0400, 0])*s
v01 = mr.ScrewToAxis(q01, w01, 0)

w02=np.array([0.9989, 0.0034, 0.0005]) 
q02 = np.array([-0.0200, 1254, 0.0500])*s
v02 = mr.ScrewToAxis(q02, w02, 0)

w03=np.array([0.0178, 0.9840, -0.0010])
q03=np.array([-1257, 0.0874, 0.0099])*s
v03 = mr.ScrewToAxis(q03, w03, 0)

w04=np.array([0.9979, 0.0129, -0.0129])
q04 = np.array([0.0751, 1249, -1103])*s
v04 = mr.ScrewToAxis(q04, w04, 0)

w05=np.array([0.0009, 0.9999, 0])
q05=np.array([-1297, -0.0205, 0.0726])*s
v05 = mr.ScrewToAxis(q05, w05, 0)

xi00 = np.c_[v01, v02, v03, v04, v05]

#initial twist
xist00=np.array([0.02, -0.01, 0.01, 0.0085, 1311, 1294])
xist00[3:6] = xist00[3:6]*s

# xist00 = np.array([[1,0,0,0],
#                    [0,1,0,1311],
#                    [0,0,1,1294],
#                    [0,0,0,1]])

mean_error_before = np.zeros((10,1))
mean_error_traditional = np.zeros((10,1))

max_error_before = np.zeros((10,1))
max_error_traditional = np.zeros((10,1))


In [6]:
# for N in np.arange(10,110,10):
for N in [10]:

    P01=np.array([100, 0, 0]) * s
    
    P02=np.array([0, 100, 0]) * s
    
    P03=np.array([0, 0, 100]) * s
    
    PX=np.c_[P01, P02, P03]

    P04=np.array([-100, -100, -100]) * s

    
    gm=np.zeros((4, 4, N))
    
    gn=np.zeros((4, 4, N))
    
    Pa1=np.zeros((4, N))
    
    Pa2=np.zeros((4, N))
    
    Pa3=np.zeros((4, N))
    
    cali_joint_angles = np.random.rand(N, 5) * 2 * np.pi - np.pi

    for i in range(0,N):
        
        Pa1[:, i] = np.dot(mr.FKinSpace(se_3_exp(xist00), xi00, cali_joint_angles[i, :]), np.r_[P01, 1].reshape((4, 1)) ).reshape((4,)) + ((np.random.rand(4, 1) * 0.2 - 0.1) * s).reshape((4,))
        
        Pa2[:, i] = np.dot(mr.FKinSpace(se_3_exp(xist00), xi00, cali_joint_angles[i, :]), np.r_[P02, 1].reshape((4, 1)) ).reshape((4,)) + ((np.random.rand(4, 1) * 0.2 - 0.1) * s).reshape((4,))
        
        Pa3[:, i] = np.dot(mr.FKinSpace(se_3_exp(xist00), xi00, cali_joint_angles[i, :]), np.r_[P03, 1].reshape((4, 1)) ).reshape((4,)) + ((np.random.rand(4, 1) * 0.2 - 0.1) * s).reshape((4,))
        
        PY = np.c_[Pa1[0:3, i], Pa2[0:3, i], Pa3[0:3, i]]

        R,t = registration(PX, PY)

        gm[0:3, 0:3, i] = R
        
        gm[0:3, 3, i] = t
        
        gn[:, :, i] = mr.FKinSpace(se_3_exp(xist00), xi00, cali_joint_angles[i, :])


    xiTFP, xistTFP = traditional_calibration_5dof(xi0, xist00, cali_joint_angles, np.round(gm, 4), 10)


    Nt=40

    error_before=np.zeros((Nt,1))
        
    error_afterTraditional=np.zeros((Nt,1))

    test_joint_config = np.random.rand(Nt,5) * 2 * np.pi - np.pi

    for i in range(0,Nt):
        
        error_before[i] = np.linalg.norm( np.dot( mr.FKinSpace(se_3_exp(xist0), xi0, test_joint_config[i,:]) - mr.FKinSpace(se_3_exp(xist00), xi00, test_joint_config[i,:]), np.r_[P04,1])) / s
        
        error_afterTraditional[i] = np.linalg.norm( np.dot( mr.FKinSpace(se_3_exp(xistTFP), xiTFP, test_joint_config[i,:]) - mr.FKinSpace(se_3_exp(xist00), xi00, test_joint_config[i,:]), np.r_[P04,1]))  / s

    idx = int(N/10) - 1

    mean_error_before[idx, 0] = np.mean(error_before)
    mean_error_traditional[idx, 0] = np.mean(error_afterTraditional)



In [7]:
print("Mean error before: ", mean_error_before)
print("Mean error traditional: ", mean_error_traditional)

Mean error before:  [[3020.262607884953]
 [   0.            ]
 [   0.            ]
 [   0.            ]
 [   0.            ]
 [   0.            ]
 [   0.            ]
 [   0.            ]
 [   0.            ]
 [   0.            ]]
Mean error traditional:  [[46.09265432271971]
 [ 0.              ]
 [ 0.              ]
 [ 0.              ]
 [ 0.              ]
 [ 0.              ]
 [ 0.              ]
 [ 0.              ]
 [ 0.              ]
 [ 0.              ]]
